# Tiền xử lí (full script)
#### <span style="color: gold;"> Thầy/cô chỉ cần chạy cell này</span>

In [2]:
# Tiền xử lí và lưu dữ liệu đã xử lí vào /data/processed_data_4/9_classes.csv
import pandas as pd

import os 
BASE_DIR = os.path.dirname(
    os.path.dirname(
        os.path.dirname(
            os.path.abspath('notebook/preprocessing.ipynb'))))

path18 = os.path.join(BASE_DIR, 'data\\raw\\ground_water_quality_2018_post.csv')
path19 = os.path.join(BASE_DIR, 'data\\raw\\ground_water_quality_2019_post.csv')
path20 = os.path.join(BASE_DIR, 'data\\raw\\ground_water_quality_2020_post.csv')

df_18 = pd.read_csv(path18)
df_19 = pd.read_csv(path19)
df_20 = pd.read_csv(path20)
print('✅ Dữ liệu đã được tải thành công!')

df_19.rename(columns={'EC': 'E.C', 'CO_-2 ': 'CO3', 'HCO_ - ': 'HCO3', 'Cl -': 'Cl', 'F -': 'F', 'NO3- ': 'NO3', 'SO4-2': 'SO4', 'Na+': 'Na', 'K+': 'K', 'Ca+2': 'Ca', 'Mg+2': 'Mg'}, inplace=True)
df_18.rename(columns={'NO3 ': 'NO3'}, inplace=True)
df_20.rename(columns={'NO3 ': 'NO3'}, inplace=True)
print('✅ Chuẩn hóa tên cột thành công!')

df_20 = pd.concat(
    [df_20.iloc[:, :23], 
     df_20['Classification'].replace('O.G', 'OG'), 
     df_20.iloc[:, 24:]
    ], 
    axis=1)
print('✅ Chuẩn hóa giá trị trong cột Classification thành công!')

df = pd.concat([df_18, df_19, df_20], ignore_index=True)
df = df.drop(columns=['CO3', 'E.C', 'sno', 'district', 'mandal', 'village', 'lat_gis', 'long_gis', 'season', 'RSC  meq  / L', 'Classification.1']).copy()
print('✅ Xóa các cột không liên quan thành công!')

gwl_mean = df['gwl'].mean()
df.fillna({'gwl': gwl_mean}, inplace=True)
print('✅ Fill giá trị null trong cột gwl bằng mean!')

df_4_classes = df[df['Classification'].isin(['C4S2', 'C4S1', 'C3S1', 'C2S1'])].copy()
df_9_classes = df[~df['Classification'].isin(['C3S4', 'C2S2', 'C1S1'])].copy()
print('✅ Tách dữ liệu thành 2 loại: 4 classes và 9 classes!')

df_4_classes.to_csv(os.path.join(BASE_DIR, 'data/processed_data_4_classes.csv'), index=False)
df_9_classes.to_csv(os.path.join(BASE_DIR, 'data/processed_data_9_classes.csv'), index=False)
print('✅ Lưu dữ liệu đã xử lí!')

✅ Dữ liệu đã được tải thành công!
✅ Chuẩn hóa tên cột thành công!
✅ Chuẩn hóa giá trị trong cột Classification thành công!
✅ Xóa các cột không liên quan thành công!
✅ Fill giá trị null trong cột gwl bằng mean!
✅ Tách dữ liệu thành 2 loại: 4 classes và 9 classes!
✅ Lưu dữ liệu đã xử lí!


# Các thử nghiệm

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!mkdir data

!wget "https://raw.githubusercontent.com/Dr-Vphuc/Ground_Water_Classifier/main/data/ground_water_quality_2018_post.csv" -O data/ground_water_quality_2018_post.csv
!wget "https://raw.githubusercontent.com/Dr-Vphuc/Ground_Water_Classifier/main/data/ground_water_quality_2019_post.csv" -O data/ground_water_quality_2019_post.csv
!wget "https://raw.githubusercontent.com/Dr-Vphuc/Ground_Water_Classifier/main/data/ground_water_quality_2020_post.csv" -O data/ground_water_quality_2020_post.csv

!ls data/

In [55]:
import os 
BASE_DIR = os.path.dirname(
    os.path.dirname(
        os.path.dirname(
            os.path.abspath('notebook/preprocessing.ipynb'))))

path18 = os.path.join(BASE_DIR, 'data\\ground_water_quality_2018_post.csv')
path19 = os.path.join(BASE_DIR, 'data\\ground_water_quality_2019_post.csv')
path20 = os.path.join(BASE_DIR, 'data\\ground_water_quality_2020_post.csv')

In [56]:
df_18 = pd.read_csv(path18)
df_19 = pd.read_csv(path19)
df_20 = pd.read_csv(path20)

## Chuẩn hóa tên cột

In [57]:
df_19.rename(columns={'EC': 'E.C', 'CO_-2 ': 'CO3', 'HCO_ - ': 'HCO3', 'Cl -': 'Cl', 'F -': 'F', 'NO3- ': 'NO3', 'SO4-2': 'SO4', 'Na+': 'Na', 'K+': 'K', 'Ca+2': 'Ca', 'Mg+2': 'Mg'}, inplace=True)
df_18.rename(columns={'NO3 ': 'NO3'}, inplace=True)
df_20.rename(columns={'NO3 ': 'NO3'}, inplace=True)

## Chuẩn hóa giá trị

### Thống nhất class O.G năm 2020 và OG năm 2019 là OG

In [58]:
df_20 = pd.concat(
    [df_20.iloc[:, :23], 
     df_20['Classification'].replace('O.G', 'OG'), 
     df_20.iloc[:, 24:]
    ], 
    axis=1)

### Xử lí null

In [59]:
# remove null values
df_18.dropna(inplace=True)
df_19.dropna(inplace=True)
df_20.dropna(inplace=True)

In [60]:
len(df_18), len(df_19), len(df_20)

(371, 202, 365)

In [61]:
df = pd.concat([df_18, df_19, df_20], ignore_index=True)

In [62]:
len(df[df['CO3'] == 0.0]), len(df)

(774, 938)

// Sự phân chia của các giá trị CO3 khác 0 vào các class không có gì đặc biệt
// mean không quá xa trong khi đo lệch chuẩn khá lớn, không có gì đặc biệt (CELL dưới)
// Vậy drop CO3 thì sẽ dữ được 160 bản ghi mà CO3 bị null, 
// bỏ E.C vì có tương quan cao với TDS

In [64]:
pd.concat([
    df[df['CO3'] > 0.0]['Classification'].value_counts(),
    df.groupby('Classification').size(),
    (df[df['CO3'] > 0.0]['Classification'].value_counts() / df.groupby('Classification').size()) * 100]
    , axis = 1)  

,count,0,1
Classification,,,
C3S1,82,584,14.041096
C2S1,53,213,24.882629
C4S4,6,7,85.714286
C4S2,5,33,15.151515
C4S1,5,72,6.944444
C3S2,4,11,36.363636
C3S3,4,6,66.666667
C1S1,1,2,50.000000
C3S4,1,1,100.000000


In [65]:
df[(df['CO3'] > 0.0) & (df['Classification'] == 'C2S1')]['CO3'].mean()
#df[(df['CO3'] > 0.0) & (df['Classification'] == 'C3S1')]['CO3'].std()

np.float64(35.92516981132076)

In [66]:
len(df[df['Classification'] == 'C2S1'])

213

In [68]:
df.groupby('Classification').size()

Classification
C1S1      2
C2S1    213
C2S2      1
C3S1    584
C3S2     11
C3S3      6
C3S4      1
C4S1     72
C4S2     33
C4S3      4
C4S4      7
OG        4
dtype: int64